In [1]:
import pypowsybl.network as pn
network1 = pn.load('/home/dupuyflo/Data/LFE_2025/fr_1850.biidm')
network2 = pn.load('/home/dupuyflo/Data/LFE_2025/fr_1900.biidm')

In [2]:
import pypowsybl.loadflow as lf
lf.run_ac(network1)

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=Converged, iteration_count=7, reference_bus_id='CHAFFP7_0', slack_bus_results=[SlackBusResult(id='CHAFFP7_0', active_power_mismatch=-0.5088339270259201)], distributed_active_power=28.570841706134765)]

In [3]:
lf.run_ac(network2)

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=Converged, iteration_count=7, reference_bus_id='CHAFFP7_0', slack_bus_results=[SlackBusResult(id='CHAFFP7_0', active_power_mismatch=-0.5743598570195352)], distributed_active_power=28.229321211999693)]

In [4]:
from pypowsybl_jupyter import network_explorer
network_explorer(network1, depth=2, vl_id='STRASP3', nad_parameters=pn.NadParameters(injections_added=True))

In [5]:
branches1 = network1.get_branches(attributes=['p1', 'p2'])
branches1

,p1,p2
id,,
.CTLHL31.CTLO,NaN,NaN
.CTLHL32.CTLO,NaN,NaN
.CTLOL31FINS,10.099362,-10.044028
.CTLOL31ZLIEB,-10.099362,10.199464
.G.ROL51HOSPI,0.000000,0.000023
...,...,...
WARANY763,-244.487931,244.680044
WARANY764,-237.615525,237.793333
WEPPEY761,-203.338081,203.466271


In [6]:
branches2 = network2.get_branches(attributes=['p1', 'p2'])
branches2

,p1,p2
id,,
.CTLHL31.CTLO,NaN,NaN
.CTLHL32.CTLO,NaN,NaN
.CTLOL31FINS,10.510379,-10.450125
.CTLOL31ZLIEB,-10.510379,10.619313
.G.ROL51HOSPI,0.000000,0.000023
...,...,...
WARANY763,-249.333843,249.532736
WARANY764,-242.326396,242.510478
WEPPEY761,-207.645618,207.778965


In [61]:
import pandas as pd
delta_p = pd.DataFrame()
branches1.fillna(0, inplace=True)
branches2.fillna(0, inplace=True)
delta_p['delta_p1'] = abs(branches2['p1']) - abs(branches1['p1'])
delta_p['delta_p2'] = abs(branches2['p2']) - abs(branches1['p2'])
delta_p

,delta_p1,delta_p2
id,,
.CTLHL31.CTLO,0.000000,0.000000e+00
.CTLHL32.CTLO,0.000000,0.000000e+00
.CTLOL31FINS,0.411016,4.060968e-01
.CTLOL31ZLIEB,0.411016,4.198489e-01
.G.ROL51HOSPI,0.000000,1.137859e-08
...,...,...
WARANY763,4.845912,4.852692e+00
WARANY764,4.710871,4.717146e+00
WEPPEY761,4.307537,4.312694e+00


In [62]:
delta_p['delta_p1'].min(), delta_p['delta_p1'].max()

(np.float64(-69.88298216782897), np.float64(103.24684411231583))

In [63]:
max_row = delta_p.loc[delta_p['delta_p1'].idxmax()]
max_row

delta_p1    103.246844
delta_p2    103.159368
Name: MOUGUL61TARN5, dtype: float64

In [88]:
import numpy as np
import pandas as pd

def create_style_bigradient(n_colors: int = 101) -> pd.DataFrame:
    center = (n_colors - 1) // 2

    colors = np.empty(n_colors, dtype='<U7')
    colors[:center-2] = '#0000ff'
    colors[center-2:center+2] = 'gray'
    colors[center+3:] = '#ff0000'

    width_neg = np.linspace(50, 1, center)                   # indices 0..49
    width_zero = np.array([1.0])                           # index 50
    width_pos = np.linspace(1, 50, center)                   # indices 51..100
    width_num = np.concatenate([width_neg, width_zero, width_pos])
    width = np.char.add(width_num.astype(int).astype(str), 'px')

    return pd.DataFrame({'hex_color': colors, 'width': width})

def map_index(value: float, n_colors: int)  -> int:
    center = (n_colors - 1) // 2
    v = float(np.clip(value, -50.0, 50.0))
    if v < 0:
        return int(round((center - 1) * (1 - abs(v) / 50.0)))
    if v > 0:
        return int(round(center + (center * (v / 50.0))))
    return center

gradient_size = 101
style = create_style_bigradient(gradient_size)
idx = pd.DataFrame({
    'idx_1': delta_p['delta_p1'].apply(lambda x: map_index(x, gradient_size)),
    'idx_2': delta_p['delta_p2'].apply(lambda x: map_index(x, gradient_size)),
})

edges_styles_df = pd.DataFrame({
    'edge1': idx['idx_1'].apply(lambda idx: style['hex_color'].iloc[idx]),
    'edge2': idx['idx_2'].apply(lambda idx: style['hex_color'].iloc[idx]),
    'width1': idx['idx_1'].apply(lambda idx: style['width'].iloc[idx]),
    'width2': idx['idx_2'].apply(lambda idx: style['width'].iloc[idx]),
})
edges_styles_df

,edge1,edge2,width1,width2
id,,,,
.CTLHL31.CTLO,gray,gray,1px,1px
.CTLHL32.CTLO,gray,gray,1px,1px
.CTLOL31FINS,gray,gray,1px,1px
.CTLOL31ZLIEB,gray,gray,1px,1px
.G.ROL51HOSPI,gray,gray,1px,1px
...,...,...,...,...
WARANY763,#ff0000,#ff0000,5px,5px
WARANY764,#ff0000,#ff0000,5px,5px
WEPPEY761,#ff0000,#ff0000,4px,4px


In [89]:
metadata = pd.read_json('/home/dupuyflo/Documents/LFE_2025/france_metadata.json')
fixed_positions = pd.DataFrame({
    "id": metadata["equipmentId"],
    "x": metadata["x"],
    "y": metadata["y"]
})
fixed_positions.set_index('id', drop=True, inplace=True)
fixed_positions

,x,y
id,,
.CTLHP3,21643.45,-140477.91
.CTLOP3,22073.05,-140487.02
.G.ROP5,3716.32,-122101.20
.G.ROP6,3723.98,-121206.74
.NAVAP3,28524.29,-127163.51
...,...,...
.ENSDL71VIGY,18020.45,-149029.21
.ENSDL72VIGY,17480.23,-149366.71
.VENAL71VLARO,19278.83,-132401.36


In [90]:
default_pf = network1.get_default_nad_profile()
vl_descriptions_df = default_pf.vl_descriptions[default_pf.vl_descriptions['type'] != 'FOOTER']
labels_df = pd.DataFrame({
    'side1': delta_p['delta_p1'].map(lambda x: f"{x:.1f}"),
    'side2': delta_p['delta_p2'].map(lambda x: f"{x:.1f}")
})

diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=default_pf.bus_descriptions,
                                      bus_node_styles=default_pf.bus_node_styles, edge_styles=edges_styles_df)

In [91]:


network_explorer(network1, depth=2, vl_id='STRASP3', nad_profile=diagram_profile, fixed_nad_positions=fixed_positions)